# Modeling

In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import classification_report, accuracy_score

import main
import prepare

## Read in the dataset

In [2]:
df = pd.read_json('../../data/raw/readme_data200.json').dropna()

In [3]:
data = prepare.prep_article_data(df, 'readme')

In [4]:
df.head()

,language,readme,watchers,stars,forks,commits,clean,stemmed,lemmatized
0,JavaScript,"FCC League-For-Good\nThis is a free, open-sour...",28,161,98,411,fcc leagueforgood free opensource web applicat...,fcc leagueforgood thi is a free opensourc web ...,fcc leagueforgood this is a free opensource we...
1,JavaScript,nba\nNode.js client for nba.com API endpoints\...,49,621,150,411,nba nodejs client nbacom api endpoint npm inst...,nba nodej client for nbacom api endpoint npm i...,nba nodejs client for nbacom api endpoint npm ...
2,JavaScript,SportsLeague: Laravel 5.4 based system for var...,6,26,48,411,sportsleague laravel 54 based system various s...,sportsleagu laravel 54 base system for variou ...,sportsleague laravel 54 based system for vario...
3,JavaScript,Team Colors\n\nTeam Colors is a reference of H...,11,123,58,411,team color team color reference hex rgb cmyk p...,team color team color is a refer of hex rgb cm...,team color team color is a reference of hex rg...
4,JavaScript,vue-sports\n\nA Vue.js project\n\n仿凤凰新闻体育板块+赛事...,5,93,42,411,vuesports vuejs project vuex vuex build setup ...,vuesport a vuej project vuex vuex build setup ...,vuesports a vuejs project vuex vuex build setu...


In [5]:
data.head()

,language,readme,stemmed,lemmatized,clean
0,JavaScript,"FCC League-For-Good\nThis is a free, open-sour...",fcc leagueforgood thi is a free opensourc web ...,fcc leagueforgood this is a free opensource we...,fcc leagueforgood free opensource web applicat...
1,JavaScript,nba\nNode.js client for nba.com API endpoints\...,nba nodej client for nbacom api endpoint npm i...,nba nodejs client for nbacom api endpoint npm ...,nba nodejs client nbacom api endpoint npm inst...
2,JavaScript,SportsLeague: Laravel 5.4 based system for var...,sportsleagu laravel 54 base system for variou ...,sportsleague laravel 54 based system for vario...,sportsleague laravel 54 based system various s...
3,JavaScript,Team Colors\n\nTeam Colors is a reference of H...,team color team color is a refer of hex rgb cm...,team color team color is a reference of hex rg...,team color team color reference hex rgb cmyk p...
4,JavaScript,vue-sports\n\nA Vue.js project\n\n仿凤凰新闻体育板块+赛事...,vuesport a vuej project vuex vuex build setup ...,vuesports a vuejs project vuex vuex build setu...,vuesports vuejs project vuex vuex build setup ...


## Transform text data using TF-IDF

In [27]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df.readme)
X = pd.concat([df[['watchers', 'stars', 'forks', 'commits']], pd.DataFrame(X_tfidf.todense())], axis=1)
y = df.language

In [88]:
repo_stats = X.iloc[:, :4]

## Split the data

In [74]:
X_train_validate, X_test, y_train_validate, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=1)
X_train, X_validate, y_train, y_validate = train_test_split(X_train_validate, y_train_validate, stratify=y_train_validate, test_size=0.25, random_state=1)

In [75]:
print(X_train.shape)
print(y_train.shape)
print(X_validate.shape)
print(y_validate.shape)
print(X_test.shape)
print(y_test.shape)

(119, 11894)
(119,)
(40, 11894)
(40,)
(40, 11894)
(40,)


## Cross Validation

In [76]:
# pd.DataFrame(LogisticRegression().get_params().keys()).T

In [77]:
# pd.DataFrame(RidgeClassifierCV().get_params().keys()).T

In [78]:
# pd.DataFrame(SVC().get_params().keys()).T

In [79]:
# pd.DataFrame(GradientBoostingClassifier().get_params().keys()).T

In [80]:
# pd.DataFrame(GaussianNB().get_params().keys()).T

In [81]:
parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

In [82]:
grid_search = GridSearchCV(LogisticRegression(), parameters, cv=10)

In [83]:
grid_search.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1344, in fit
    accept_large_sparse=solver != 'liblinear')
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 72, in inner_f
    return f(**kwargs)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 802, in check_X_y
    estimator=estimato

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1344, in fit
    accept_large_sparse=solver != 'liblinear')
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 72, in inner_f
    return f(**kwargs)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 802, in check_X_y
    estimator=estimato

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1344, in fit
    accept_large_sparse=solver != 'liblinear')
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 72, in inner_f
    return f(**kwargs)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 802, in check_X_y
    estimator=estimato

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1344, in fit
    accept_large_sparse=solver != 'liblinear')
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 72, in inner_f
    return f(**kwargs)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 802, in check_X_y
    estimator=estimato

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1344, in fit
    accept_large_sparse=solver != 'liblinear')
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 72, in inner_f
    return f(**kwargs)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 802, in check_X_y
    estimator=estimato

KeyboardInterrupt: 

In [ ]:
grid_search.score(X_test, y_test)

In [ ]:
grid_search.best_estimator_

In [ ]:
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

lm = SVC(C=100, gamma=0.1).fit(X_train, y_train)

train['predicted'] = lm.predict(X_train)
test['predicted'] = lm.predict(X_test)

# Evaluate

In [ ]:
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.predicted, train.actual))
print('---')
print(classification_report(train.actual, train.predicted))

In [ ]:
print('Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(test.predicted, test.actual))
print('---')
print(classification_report(test.actual, test.predicted))